In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,ExtraTreesRegressor , RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
from sklearn.neural_network import MLPClassifier

In [15]:
df = pd.read_csv('submission files/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              2
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
df.dropna(subset=['Age'], inplace=True) # drop rows with missing ages 

In [19]:
df['Has_Cabin'] = df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [20]:
df.drop(columns= ['PassengerId', 'Name', 'Cabin', 'Ticket', 'Embarked'], inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Has_Cabin
0,0,3,male,22.0,1,0,7.2500,0
1,1,1,female,38.0,1,0,71.2833,1
2,1,3,female,26.0,0,0,7.9250,0
3,1,1,female,35.0,1,0,53.1000,1
4,0,3,male,35.0,0,0,8.0500,0


In [21]:
y = df.iloc[:, 0].values
X = df.iloc[:,1:].values
X

array([[3, 'male', 22.0, ..., 0, 7.25, 0],
       [1, 'female', 38.0, ..., 0, 71.2833, 1],
       [3, 'female', 26.0, ..., 0, 7.925, 0],
       ...,
       [3, 'female', 7.0, ..., 2, 23.45, 0],
       [1, 'male', 26.0, ..., 0, 30.0, 1],
       [3, 'male', 32.0, ..., 0, 7.75, 0]], dtype=object)

In [22]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[0.0 1.0 3 ... 0 7.25 0]
 [1.0 0.0 1 ... 0 71.2833 1]
 [1.0 0.0 3 ... 0 7.925 0]
 ...
 [1.0 0.0 3 ... 2 23.45 0]
 [0.0 1.0 1 ... 0 30.0 1]
 [0.0 1.0 3 ... 0 7.75 0]]


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[0.0 1.0 3 ... 0 8.05 0]
 [0.0 1.0 3 ... 0 7.7958 0]
 [0.0 1.0 2 ... 0 26.0 0]
 ...
 [0.0 1.0 3 ... 0 14.4542 0]
 [0.0 1.0 2 ... 0 26.0 0]
 [0.0 1.0 3 ... 0 8.05 0]]
[[1.0 0.0 2 ... 0 13.0 0]
 [1.0 0.0 2 ... 1 26.0 0]
 [1.0 0.0 2 ... 0 10.5 1]
 ...
 [0.0 1.0 1 ... 0 135.6333 0]
 [0.0 1.0 3 ... 0 8.05 0]
 [0.0 1.0 2 ... 0 10.5 0]]
[0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1
 1 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 0
 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0
 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0
 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1
 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 

In [24]:
sc = StandardScaler()
X_train[:, 2:] = sc.fit_transform(X_train[:, 2:])
X_test[:, 2:] = sc.transform(X_test[:, 2:])
print(X_train)
print(X_test)

[[0.0 1.0 0.8186003825681544 ... -0.4819083957625295 -0.4852246657387601
  -0.5410427642315099]
 [0.0 1.0 0.8186003825681544 ... -0.4819083957625295 -0.49046325581272293
  -0.5410427642315099]
 [0.0 1.0 -0.3914305363631511 ... -0.4819083957625295
  -0.11530849017765835 -0.5410427642315099]
 ...
 [0.0 1.0 0.8186003825681544 ... -0.4819083957625295 -0.3532459932302137
  -0.5410427642315099]
 [0.0 1.0 -0.3914305363631511 ... -0.4819083957625295
  -0.11530849017765835 -0.5410427642315099]
 [0.0 1.0 0.8186003825681544 ... -0.4819083957625295 -0.4852246657387601
  -0.5410427642315099]]
[[1.0 0.0 -0.3914305363631511 ... -0.4819083957625295 -0.3832143554865343
  -0.5410427642315099]
 [1.0 0.0 -0.3914305363631511 ... 0.7965873855328378 -0.11530849017765835
  -0.5410427642315099]
 [1.0 0.0 -0.3914305363631511 ... -0.4819083957625295
  -0.43473471419977966 1.848282734952363]
 ...
 [0.0 1.0 -1.6014614552944566 ... -0.4819083957625295 2.144030286989079
  -0.5410427642315099]
 [0.0 1.0 0.81860038256

In [25]:
classifierlg = LogisticRegression(random_state = 1)
classifierlg.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [26]:
y_pred = classifierlg.predict(X_test)
print(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1.0 0.0 -0.3914305363631511 ... -0.4819083957625295 -0.3832143554865343
  -0.5410427642315099]
 [1.0 0.0 -0.3914305363631511 ... 0.7965873855328378 -0.11530849017765835
  -0.5410427642315099]
 [1.0 0.0 -0.3914305363631511 ... -0.4819083957625295
  -0.43473471419977966 1.848282734952363]
 ...
 [0.0 1.0 -1.6014614552944566 ... -0.4819083957625295 2.144030286989079
  -0.5410427642315099]
 [0.0 1.0 0.8186003825681544 ... -0.4819083957625295 -0.4852246657387601
  -0.5410427642315099]
 [0.0 1.0 -0.3914305363631511 ... -0.4819083957625295
  -0.43473471419977966 -0.5410427642315099]]
[[1 1]
 [1 1]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [

0.8089887640449438

In [27]:
filename = 'LogisticRegression_model808.sav'
pickle.dump(classifierlg, open(filename, 'wb'))

In [28]:
classifierknn = KNeighborsClassifier(n_neighbors = 5)
classifierknn.fit(X_train, y_train)

KNeighborsClassifier()

In [29]:
y_pred = classifierknn.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[92 13]
 [22 51]]


0.8033707865168539

In [32]:
model = MLPClassifier(hidden_layer_sizes=(32,16,1),
                      solver='adam', alpha= 0.001, learning_rate='adaptive',max_iter = 1000,
                      activation='relu')
model.fit(X_train,y_train)
predictions = model.predict(X_test)
cm = confusion_matrix(y_test, predictions)
print(cm)
accuracy_score(y_test, predictions)
print('Accuracy score ',accuracy_score(y_test,predictions))

[[96  9]
 [20 53]]
Accuracy score  0.8370786516853933


In [33]:
filename = 'MLPClassifier_model837.sav'
pickle.dump(model, open(filename, 'wb'))

In [34]:
classifierdct = DecisionTreeClassifier(random_state = 0)
classifierdct.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [35]:
y_pred = classifierdct.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[87 18]
 [31 42]]


0.7247191011235955

In [36]:
classifierf = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 1)
classifierf.fit(X_train, y_train)
y_pred = classifierf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[96  9]
 [32 41]]


0.7696629213483146

In [37]:
classifierxgb = XGBClassifier()
classifierxgb.fit(X_train, y_train)
y_pred = classifierxgb.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1 1]
 [1 1]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [1 0]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 1]
 [0 1]
 [0 0]
 [0 0]

0.797752808988764